In [1]:
import PyPDF2
import pandas as pd
import fasttext
from io import StringIO
import re
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
from nltk import sent_tokenize
from nltk.corpus import stopwords



In [ ]:
def get_grading_policy(input_df):
    grading_df = input_df.loc[input_df['has_grading_info'] == 1]

    f_index = grading_df.head(1).index[0]
    l_index = grading_df.tail(1).index[0]
        
    num_categories = len(grading_df)   
    percentages = []
    descriptions = []
    
    values = string.split('%', num_categories)  
    
    for i in range(num_categories):
        percent_value = values[i][-2:] + "%"
        if i != num_categories - 1: 

            description = values[i+1][:-3]
        else: 
            description = values[i+1]
        
        percentages.append(percentages)
        descriptions.append(description)

#         print(percent_value)
#         print(description)
    
    return_df = pd.DataFrame()
    return_df['percentage'] = percentages
    return_df['description'] = descriptions
    
    return return_df
    